In this code example, we'll apply feedback control to stabilize a 2D quadrotor in the presence of significant disturbances.

In [1]:
%pylab inline
from pylab import *

import control
import matplotlib.animation
import scipy.interpolate

rc("animation", html="html5")

import path_utils
path_utils.add_relative_to_current_source_file_path_to_sys_path("../../lib")

import flashlight.curve_utils       as curve_utils
import flashlight.interpolate_utils as interpolate_utils
import flashlight.quadrotor_2d      as quadrotor_2d
import flashlight.spline_utils      as spline_utils
import flashlight.trig_utils        as trig_utils

Populating the interactive namespace from numpy and matplotlib
Initializing flashlight v0.0.1
flashlight.quadrotor_2d: Constructing sympy symbols...
flashlight.quadrotor_2d: Finished constructing sympy symbols (0.004 seconds).
flashlight.quadrotor_2d: Loading sympy modules...
flashlight.quadrotor_2d: Finished loading sympy modules (0.002 seconds).


/Users/mike/Code/GitHub/flashlight/code/lib/flashlight/transformations.py:1888: UserWarning: failed to import module _transformations
  warnings.warn("failed to import module %s" % name)


We begin by defining the point in state-control space about which we want to linearize the quadrotor dynamics. In this example, we choose to linearize the dynamics about hovering-at-the-origin. To linearize the quadrotor dynamics, we call `quadrotor_2d.compute_df_dx_and_df_du` and pass in our linearization point. To compute the control forces required for hovering, we must use the mass and gravitational acceleration constants assumed by our quadrotor dynamics model. We access these constants using the `quadrotor_2d.m` and `quadrotor_2d.g` variables.

Next, we must also specify the cost matrices that penalize deviations away from our hover-at-the-origin point in state-control space.

After having defined these matrices `(A,B,Q,R)`, we can compute the optimal control matrix `K`, such that the optimal control force `u = -K*x` for all states `x`, under the assumption that our stabilization point is the origin in state-control space. For other stabilization points, we simply translate our coordinate system so that our stabilization point is the origin. For more details on the underlying optimal control problem being solved here, see [1].

#### References

[1] Russ Tedrake. Underactuated Robotics. Algorithms for Walking, Running, Swimming, Flying, and Manipulation (course notes for MIT 6.832). http://underactuated.mit.edu

In [2]:
m = quadrotor_2d.m
g = quadrotor_2d.g

x_star = matrix([0.0,0.0,0.0,0.0,0.0,0.0]).T
u_star = matrix([m*g/2.0,m*g/2.0]).T

Q = diag([1,1,1,1,1,1])
R = diag([1,1])

A, B = quadrotor_2d.compute_df_dx_and_df_du(x_star, u_star)

K, S, E = control.lqr(A, B, Q, R)

print "A ="; print A; print; print "B ="; print B; print; print "K ="; print K

A =
[[ 0.   0.   0.   1.   0.   0. ]
 [ 0.   0.   0.   0.   1.   0. ]
 [ 0.   0.   0.   0.   0.   1. ]
 [ 0.   0.  -0.   0.   0.   0. ]
 [ 0.   0.  -9.8  0.   0.   0. ]
 [ 0.   0.   0.   0.   0.   0. ]]

B =
[[ 0.  0.]
 [ 0.  0.]
 [ 0.  0.]
 [ 1.  1.]
 [-0. -0.]
 [-1.  1.]]

K =
[[ 0.70710678  0.70710678 -8.34815843  1.09868411  1.30564234 -2.97458542]
 [ 0.70710678 -0.70710678  8.34815843  1.09868411 -1.30564234  2.97458542]]


Next, we set up our simulation callback function to compute the appropriate control forces and compute the time derivative of our quadrotor state. We include a bit of extra logic here to handle the fact that the `theta` dimension of our state space wraps around. So if our ideal orientation is `theta=0.0` and we are currently at `theta=1.99*pi`, we would like for this to be considered a small error.

In [3]:
p_star, theta_star, p_dot_star, theta_dot_star, q_star, q_dot_star = quadrotor_2d.unpack_state(x_star)

def compute_x_dot(x_t, t):

    x_t = matrix(x_t).T
    p_t, theta_t, p_dot_t, theta_dot_t, q_t, q_dot_t = quadrotor_2d.unpack_state(x_t)

    theta_star_handles_wrap_t = theta_t + trig_utils.compute_smallest_angular_diff(theta_t, theta_star)
    x_star_handles_wrap_t     = quadrotor_2d.pack_state(p_star, theta_star_handles_wrap_t, p_dot_star, theta_dot_star)
    
    x_bar_t = x_t - x_star_handles_wrap_t
    u_bar_t = -K*x_bar_t
    u_t     = u_bar_t + u_star
    x_dot_t = quadrotor_2d.compute_x_dot(x_t, u_t).A1
    
    return x_dot_t

num_timesteps_sim = 200

t_begin = 0.0
t_end   = 10.0
t_sim   = linspace(t_begin, t_end, num_timesteps_sim)
dt_sim  = (t_end-t_begin) / num_timesteps_sim

Finally, we specify our initial state to be some distance away from our linearization point, with a large initial velocity, to examine the behavior of our LQR feedback control strategy.

In [4]:
x_disturbance = matrix([-1.0,-1.0,pi/6.0,5.0,-5.0,-4.0*pi]).T
x_0           = (x_star + x_disturbance).A1
x_sim         = scipy.integrate.odeint(compute_x_dot, x_0, t_sim)

figsize(6,4)
quadrotor_2d.draw(t_sim, x_sim, return_anim_func=True)